<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/threadLayout_Indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.1 1,024보다 큰 벡터의 합 구하기

1.스레드 레이아웃 결정

- 절차 : 블록 크기 결정 -> 데이터의 크기 및 블록 크기에 따라 그리드 크기 결정
- 블록 크기 결정 시에는 커널의 성능 특성과 GPU 자원의 제한을 고려해야함
- 블록 및 그리드 크기와 레지스터나 공유 메모리 크기 등도 제한이 있는 자원에 속함

2.각 스레드가 접근할 데이터 인덱스 계산
- threadIdx만 사용하서 벡터 내 원소를 접근하는 경우 : 여러 개의 블록을 사용하는 경우 한 블록 내에서는 모두 스레드 번호가 다르지만, 다른 블록에는 번호가 같은 스레드가 존재하여 모든 블록의 n번 스레드는 동일한 원소에 접근하게 됨

=> 우리는 1번 블록의 m번째 스레드는 vector[블록의 크기+m]원소를 처리하도록 해야함

**1차원 블록의 경우 m번째 스레드가 접근할 벡터의 원소**



```
vector[blockDim.x + threadIdx.x]
```

**n번 블록 m번째 스레드가 접근할 벡터의 원소**


```
vector[blockIdx.x*blockDim.x+threadIdx.x]
```


3.계산된 인덱스를 반영한 커널 작성

In [ ]:
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <chrono>
#include <iostream>

// The size of the vector
#define NUM_DATA (1024*1024*128)

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int* _a, int* _b, int* _c, int _size) {
	int tID = blockIdx.x*blockDim.x+threadIdx.x;
  if(tID< _size)
	  _c[tID] = _a[tID] + _b[tID]; //마지막 블록의 경우 벡터의 크기를 벗어나는 인덱스를 가져 잘못된 영역에 접근하므로 벡터 크기 이상으로는 작업을 중지하도록 예외처리 필요함
}

int main(void)
{

	int* a, * b, * c, * hc;	// Vectors on the host
	int* da, * db, * dc;	// Vectors on the device

	int memSize = sizeof(int) * NUM_DATA;
	printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

	// Memory allocation on the host-side
	a = new int[NUM_DATA]; memset(a, 0, memSize);
	b = new int[NUM_DATA]; memset(b, 0, memSize);
	c = new int[NUM_DATA]; memset(c, 0, memSize);
	hc = new int[NUM_DATA]; memset(hc, 0, memSize);

	// Data generation
	for (int i = 0; i < NUM_DATA; i++) {
		a[i] = rand() % 10;
		b[i] = rand() % 10;
	}


	// Vector sum on host (for performance comparision)
  auto hostStart = std::chrono::high_resolution_clock::now();
	for (int i = 0; i < NUM_DATA; i++)
		hc[i] = a[i] + b[i];
  auto hostEnd = std::chrono::high_resolution_clock::now();
  // 밀리초 단위로 경과 시간 계산 (소수점 포함)
  std::chrono::duration<double, std::milli> hostElapsed = hostEnd - hostStart;


	// Memory allocation on the device-side
	cudaMalloc(&da, memSize); cudaMemset(da, 0, memSize);
	cudaMalloc(&db, memSize); cudaMemset(db, 0, memSize);
	cudaMalloc(&dc, memSize); cudaMemset(dc, 0, memSize);

  cudaEvent_t kernelStart, kernelStop;
  cudaEventCreate(&kernelStart);
  cudaEventCreate(&kernelStop);

  auto GPUstart = std::chrono::high_resolution_clock::now();

	// Data copy : Host -> Device
	auto h2dStart = std::chrono::high_resolution_clock::now();
	cudaMemcpy(da, a, memSize, cudaMemcpyHostToDevice);
	cudaMemcpy(db, b, memSize, cudaMemcpyHostToDevice);
	auto h2dEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> h2dElapsed = h2dEnd - h2dStart;

  //thread Layout : host 코드임
  auto threadLayoutStart = std::chrono::high_resolution_clock::now();
  dim3 dimGrid(ceil((float)NUM_DATA/256),1,1);
  dim3 dimBlock(256,1,1);
  auto threadLayoutEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> threadLayoutElapsed = threadLayoutEnd - threadLayoutStart;

	// Kernel call
	cudaEventRecord(kernelStart, 0);
	vecAdd <<<dimGrid, dimBlock >>> (da, db, dc, NUM_DATA);
  cudaEventRecord(kernelStop, 0);
  cudaEventSynchronize(kernelStop);
  float kernelTime;
  cudaEventElapsedTime(&kernelTime, kernelStart, kernelStop);


	// Copy results : Device -> Host
  auto d2hStart = std::chrono::high_resolution_clock::now();
	cudaMemcpy(c, dc, memSize, cudaMemcpyDeviceToHost);
  auto d2hEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> d2hElapsed = d2hEnd - d2hStart;

	auto GPUend = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> GPUelapsed = GPUend - GPUstart;

	// Release device memory
	cudaFree(da); cudaFree(db); cudaFree(dc);

  // 결과 출력 (밀리초 단위, 소수점 포함)
  std::cout << "Host time: " << hostElapsed.count() << " ms" << std::endl;
  std::cout<<"Host -> Device: " << h2dElapsed.count() << " ms" << std::endl;
  std::cout<<"Kernel: " << threadLayoutElapsed.count() + kernelTime << " ms" << std::endl;
  std::cout<<"Device -> Host: " << d2hElapsed.count() << " ms" << std::endl;
  std::cout<<"CUDA Total Time: " << GPUelapsed.count() << " ms" << std::endl;

  cudaEventDestroy(kernelStart);
  cudaEventDestroy(kernelStop);

	// Check results
	bool result = true;
	for (int i = 0; i < 1; i++) {
		if (hc[i] != c[i]) {
			printf("[%d] The result is not matched! (%d, %d)\n"
				, i, hc[i], c[i]);
			result = false;
		}
	}

	if (result)
		printf("GPU works well!\n");

	// Release host memory
	delete[] a; delete[] b; delete[] c; delete[] hc;

	return 0;
}

134217728 elements, memSize = 536870912 bytes
Host time: 393.343 ms
Host -> Device: 229.888 ms
Kernel: 6.29047 ms
Device -> Host: 112.496 ms
CUDA Total Time: 348.766 ms
GPU works well!



결과
- CPU연산보다 GPU연산 시 명백히 더 빠른 연산 성능을 보임

주의할 점
- 연산 시간 뿐만 아니라 데이터 전송 시간을 고려해야함
- 데이터 전송 시간 고려시 큰 차이가 나지 않음

=> 필요한 연산 양에 비해 데이터 전송 시간의 부하가 매우 커서 CPU사용 대비 효율이 낮음

=> 단순한 연산의 경우 연산 속도보다 데이터 전송 시간의 부하가 더 커질 수도 있음

- GPU 스레드 레이아웃은 GPU 환경 설정과 관련된 부분으로 CPU에서 동작함

# 5.2 스레드 인덱싱

1.메모리 속 배열의 모습
- 메모리에는 차원의 개념이 없으며, 저장 공간이 일렬고 나열된 형태임
- 물리적 메모리 공간은 1차원이며 고차원 배열도 1차원의 형태로 저장됨

2.스레드 인덱싱 연습1 - 스레드의 전역 번호
- 스레드 마다 하나의 데이터를 담당하는 경우 그리드 내에서 스레드의 전역 번호(global ID)를 만들어 사용하는 것이 일반적임

가. 블록 내 스레드의 전역 번호

1) 1차원 블록
- 1차원 블록의 경우 스레드는 x-차원의 번호만 갖기에 threadIdx.x가 스레드의 전역번호와 동일

2) 2차원 블록
- 2차원 블록은 1차원 형태의 하위 블록 여러 개가 y-차원으로 나열되어 있다고 생각
 - 자신이 속한 1차원 하위 블록 앞까지의 스레드 개수 = blockDim.x + threadIdx.y
 - 자신이 속한 1차원 하위 블록 안에서 자신의 스레드 번호 = threadIdx.x


```
2D_BLOCK_TID = (blockDim.x * threadIdx.y + threadIdx.x)
```

3) 3차원 블록
- 2차원 형태의 하위 블록 여러개가 z-차원으로 나열되어있음
 - 자신이 속한 2차원 하위 블록 앞까지의 스레드 개수 = (blockDim.x * blockDim.y)*threadIdx.z
 - 자신이 속한 2차원 하위 블록 안에서 자신의 스레드 번호 = 2D_BLOCK_TID



```
TID_IN_BLOCK = (blockDim.x *blockDIm.y *threadIdx.z + 2D_BLOCK_TID)
```



나. 그리드 내 스레드의 전역 번호
- 그리드 내 블록이 여러 개라면 각 블록 번호를 고려해서 전역 스레드 번호를 계산해야함
 - 자신이 속한 블록의 앞 블록까지의 스레드 개수
 - 자신이 속한 블록 내에서 자신이 몇 번째 스레드인가? = TID_IN_BLOCK

1) 1차원 그리드(ID grid)
- 블록 하나에 속한 스레드의 개수
 - (NUM_THREAD_IN_BLOCK) = (blockDim.z * blockDIm.y * blockDIm.x)
 - 그리드 차원과 무관하게 동일
- 자신이 속한 블록의 번호
 - blockIdx.x


```
ID_GRID_TID = (blockIdx.x * (NUM_THREAD_IN_BLOCK) + TID_IN_BLOCK
```

2) 2차원 그리드(2D grid)
- 1차원 형태의 하위 그리드 여러 개가 y-차원으로 나열되어 있다고 생각
- 1차원 하위 그리그 안의 스레드 개수
 - gridDim.x * NUM_THREAD_IN_BLOCK
- 자신이 속한 1차원 하위 그리드의 번호
 - blockIdx.y
- 자신이 속한 1차원 하위 그리드 내에서 자신의 스레드 번호
 - 1D_GRID_TID



```
2D_GRID_TID = (blockIdx.y * (gridDim.x * NUM_THREAD_IN_BLOCK) + ID_GRID_TID
```

3) 3차원 그리드(3D grid)
- 2차원 하위 그리드 하나 안의 스레드 개수
 - gridDim.y * gridDim.x * NUM_THREAD_IN_BLOCK
- 자신이 속한 2차원 하위 그리드 번호
 - blockIdx.z
- 자신이 속한 2차원 하위 그리드 내에서 자신의 스레드 번호
 - 2D_GRID_TID


```
GLOBAL_TID = (blockIdx.z * (gridDim.y * gridDim.x * NUM_THREAD_IN_BLOCK)) + 2D_GRID_TID
```




**스레드 전역 번호 사용을 위한 기호 상수 정의 예시**


```
//Block ID
#define BID_X blockIdx.x
#define BID_Y blockIdx.y
#define BID_Z blockIdx.z

//Thread ID
#define TID_X threadIdx.x
#define TID_Y threadIdx.y
#define TID_X threadIdx.z

//Dimension of a grid
#define Gdim_X gridDim.x
#define Gdim_Y gridDim.y
#define Gdim_Z gridDim.z

//Dimension of a block
#define Bdim_X blockDIm.x
#define Bdim_Y blockDIm.y
#define Bdim_Z blockDim.z

#define TID_IN_BLOCK  (TID_Z*(Bdim_Y*Bdim_X) + TID_Y*Bdim_X + TID_X)
#define NUM_THREAD_IN_BLOCK (Bdim_X*Bdim_Y*Bdim_Z)

#define GRID_ID_TID (BID_X * NUM_THREAD_IN_BLOCK) + TID_IN_BLOCK
#define GRID_2D_TID (BID_Y + (Gdim_X * NUM_THREAD_IN_BLOCK) + GRID_ID_TID)
#define GLOBAL_TID (BID_Z*(Gdim_Y*GDIM_X*NUM_THREAD_IN_BLOCK) +GRID_2D_TID)
```



3.스레드 인덱싱 연습 2 - 2차원 데이터에 대한 인덱싱
- 행렬(2차원 데이터)을 다룰 때 사용하는 대표적인 인덱싱 방법은 2차원 스레드 번호를 사용해 각 스레드가 행렬의 담당 원소를 가르키게 하는 것임

=> 2차원 형태의 스레드 레이아웃을 사용해는 것이 가장 직관적임

- 2차원 스레드 블록 사용 시 각 스레드는 (x,y)의 2차원 번호를 가짐
 - 행렬의 각 원소는 (행,열)의 2차원 번호를 가지며 스레드의 2차원 번호를 이에 매칭할 수 있음
 - 스레드 번호를 매칭할 때는 x-차원 번호화 y-차원 번호를 각각 행,열 중 어느 것에 대응시킬지를 결정해야함

1) 스레드가 담당할 행렬의 원소 결정
- 행, 열 둘 다 가능하지만 여기서는 x-차원 스레드 번호를 행렬의 열에, y-차원 스레드 번호를 행렬의 행에 대응

```
col = threadIdx.x
row = threadIdx.y

```


- 블록 크기를 행렬 크기와 일치하도록 레이아웃을 잡으면 스레드와 행렬의 원소가 1:1 매칭되는 모습이 됨

2) 해당 원소에 접근하기 위한 인덱스 계산
- 고차원 데이터도 메모리에는 1차원 형태로 저장됨

```
index (row, col) = row * (행의 길이) + col
```
- 현재 스레드 레이아웃에서 블록 크기는 행렬 크기와 일치하기에 행 길이는 블록의 x-차원 길이인 blockDim.x와 같음


```
index (row, col) = row * blockDim.x + col
                 = threadIdx.y * blockDim.x + threadIdx.x
```

3) 두 행렬의 합을 구하는 CUDA 커널 코드


```
__global__ void matADD_2D_index (float * _dA , float* _dB, float* _dC)
{
  unsigned int col = threadIdx.x;
  unsigned int row = threadIdx.y;
  unsigned int index = row * bloackDim.x + col;

  _dC[index] = _dA[index] + _dB[index];
}

//kernel call
dim3 blockDim(COL_SIZE, ROW_SIZE);
matADD_2D_index <<<1, blockDim>>> (dA, dB, dC);
```
- 하나의 블록만 사용하는 2차원 인덱싱 방법이며 행렬 크기가 블록 최대 크기(1,024)보다 작은 경우에만 정상 작동함








# 5.3 CUDA 기반 대규모 행렬 합 프로그램

- 문제 정의
 - 행과 열의 크기가 각각 1,024보다 큰 두 행렬 A,B를 더해서 행렬 C에 저장
 - 행렬의 합이므로 세 행렬의 크기는 동일하며 각 원소는 float형임
 - (2차원 그리드, 2차원 블록) , (1차원 그리드, 1차원 블록) 그리고 (2차원 그리드, 1차원 블록) 세 가지 형태의 스레드 레이아웃으로 구현

1.2차원 그리드, 2차원 블록 레이아웃

- 커널 작성
 - 그리드와 블록 모두 2차원이기 때문에 블록 번호와 스레드 번호 모두 x-차원과 y-차원의 번호를 가짐
 - 블록의 x-차원 번호와 스레드의 x-차원 번호의 조합으로 유일한 열 번호를 계산할 수 있음


```
col = (blockIdx.x * blockDIm.x) + threadIdx.x
row = (blockIdx.y * blockDim.y) + threadIdx.y

index(row, col) = row * COL_SIZE(열의 개수) + col
```

- 그리드와 블록 크기 결정
 - 블록 크기
   - 1,024를 넘을 수 없음
   - (32, 32) 크기의 2차원 블록을 사용 (예시)
 - 그리드 크기
  - 행렬의 각 행과 길이를 생각, 올림 처리
   - x차원으로 필요한 개수 : COL_SIZE / blockDIm.x
   - y차원으로 필요한 개수 : ROW_SIZE / blockDIm.y


2.1차원 그리드, 1차원 블록 레이아웃
- 스레드 개수가 행렬 크기(전체 원소 개수)와 같도록 블록과 그리드의 크기를 잡음
- 벡터의 합과 유사하게 각 스레드가 하나의 원소를 담당하도록 함
- 행렬의 크기가 커짐에 따라 필요한 스레드의 수가 급격히 증가함

 => 스레드의 전역 번호를 행렬의 행과 열 중 하나와 매칭하는 방법 사용

 - 각 스레드가 본인 담당 열에 대한 모든 행을 처리하도록 하는 방법


In [1]:
from typing import Match
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <chrono>
#include <iostream>

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

enum ThreadLayout
{
	G1D_B1D, G2D_B1D, G2D_B2D, NUM_LAYOUTS
};

/**
Interface function for kernel call
*/
bool kernelCall
(float* _MatA, float* _MatB, float* _MatC, int _nRow, int _nCol
	, int _layout, dim3 _griDim, dim3 _blockDim);


__global__ void MatAdd_G2D_B2D
(float* MatA, float* MatB, float* MatC, int nRow, int nCol)
{
	// Write your 2D_2D kernel here
 unsigned int col = threadIdx.x + blockIdx.x * blockDim.x;
 unsigned int row = threadIdx.y + blockIdx.y * blockDim.y;
 unsigned int index = row * COL_SIZE + col;

 if(col<COL_SIZE && row<ROW_SIZE) //예외 처리
    MatC[index] = MatA[index] + MatB[index];
}

__global__ void MatAdd_G1D_B1D
(float* MatA, float* MatB, float* MatC, int nRow, int nCol)
{
	// Write your 1D_1D kernel here
 unsigned int col = threadIdx.x + blockIdx.x * blockDim.x;
 if(col<COL_SIZE){ //각 스레드가 자신이 담당하는 열에 대한 모든 행을 처리하는 부분
     for(int row = 0; row<ROW_SIZE ; row++){
         int index = row* COL_SIZE + col;
         MatC[index] = MatA[index] + MatB[index];
     }
 }
}

__global__ void MatAdd_G2D_B1D
(float* MatA, float* MatB, float* MatC, int nRow, int nCol)
{
	// Write your 2D_1D kernel here
}


bool kernelCall(float* _MatA, float* _MatB, float* _MatC, int _nRow, int _nCol
	, int _layout, dim3 _gridDim, dim3 _blockDim)
{
	switch (_layout)
	{
	case ThreadLayout::G1D_B1D:
		MatAdd_G1D_B1D <<<_gridDim, _blockDim >>> (_MatA, _MatB, _MatC, _nRow, _nCol);
		break;
	case ThreadLayout::G2D_B1D:
		MatAdd_G2D_B1D <<<_gridDim, _blockDim >>> (_MatA, _MatB, _MatC, _nRow, _nCol);
		break;
	case ThreadLayout::G2D_B2D:
		MatAdd_G2D_B2D <<<_gridDim, _blockDim >>> (_MatA, _MatB, _MatC, _nRow, _nCol);
		break;
	default:
		printf("Not supported layout\n");
		return false;
	}
	return true;
}

#define ROW_SIZE (8192)
#define COL_SIZE (8192)
#define MAT_SIZE (ROW_SIZE*COL_SIZE)

#define ID2INDEX(_row,_col) (_row*COL_SIZE + _col)

bool MatAddGPU_2D2D(float* _dA, float* _dB, float* _dC);
bool MatAddGPU_1D1D(float* _dA, float* _dB, float* _dC);
bool MatAddGPU_2D1D(float* _dA, float* _dB, float* _dC);

int main(void)
{
	float *A, *B, *C[4], *hC;
	float *dA, *dB, *dC;

	//// host memory allocation
	allocNinitMem<float>(&A, MAT_SIZE);
	allocNinitMem<float>(&B, MAT_SIZE);
	LOOP_I(NUM_LAYOUTS) {
		allocNinitMem<float>(&C[i], MAT_SIZE);
	}
	allocNinitMem<float>(&hC, MAT_SIZE);

	// device memory allocation
	cudaMalloc(&dA, sizeof(float) * MAT_SIZE); cudaMemset(dA, 0, sizeof(float) * MAT_SIZE);
	cudaMalloc(&dB, sizeof(float) * MAT_SIZE); cudaMemset(dB, 0, sizeof(float) * MAT_SIZE);
	cudaMalloc(&dC, sizeof(float) * MAT_SIZE); cudaMemset(dC, 0, sizeof(float) * MAT_SIZE);

	// input matrix generation
	for (int i = 0; i < MAT_SIZE; i++) {
		A[i] = rand() % 100;
		B[i] = rand() % 100;
	}

	// Check host performamce
	auto hostStart = std::chrono::high_resolution_clock::now();
	for (int row = 0; row < ROW_SIZE; row++) {
		for (int col = 0; col < COL_SIZE; col++) {
			hC[ID2INDEX(row, col)] = A[ID2INDEX(row, col)] + B[ID2INDEX(row, col)];
		}
	}
	auto hostEnd = std::chrono::high_resolution_clock::now();
 std::chrono::duration<double, std::milli> hostElapsed = hostEnd - hostStart;

	// copy the input matrices from host memory to device memory
	auto h2dStart = std::chrono::high_resolution_clock::now();
	cudaMemcpy(dA, A, sizeof(float) * MAT_SIZE, cudaMemcpyHostToDevice);
	cudaMemcpy(dB, B, sizeof(float) * MAT_SIZE, cudaMemcpyHostToDevice);
	auto h2dEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> h2dElapsed = h2dEnd - h2dStart;

	// *** Kernel call
	auto 2D2DStart = std::chrono::high_resolution_clock::now();
	MatAddGPU_2D2D(dA, dB, dC); cudaDeviceSynchronize();
	auto 2D2DEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> 2D2DElapsed = 2D2DEnd - 2D2DStart;

  auto d2hStart = std::chrono::high_resolution_clock::now();
	cudaMemcpy(C[G2D_B2D], dC, sizeof(float)* MAT_SIZE, cudaMemcpyDeviceToHost);
	auto d2hEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> d2hElapsed = d2hEnd - d2hStart;

	auto 1D1DStart = std::chrono::high_resolution_clock::now();
	MatAddGPU_1D1D(dA, dB, dC); cudaDeviceSynchronize();
	auto 1D1DEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> 1D1DElapsed = 1D1DEnd - 1D1DStart
	cudaMemcpy(C[G1D_B1D], dC, sizeof(float)* MAT_SIZE, cudaMemcpyDeviceToHost);

	auto 2D1DStart = std::chrono::high_resolution_clock::now();
	MatAddGPU_2D1D(dA, dB, dC); cudaDeviceSynchronize();
	auto 2D1DEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> 2D1DElapsed = 2D1DEnd - 2D1DStart;
	cudaMemcpy(C[G2D_B1D], dC, sizeof(float)* MAT_SIZE, cudaMemcpyDeviceToHost);


  std::cout << "Host time: " << hostElapsed.count() << " ms" << std::endl;
  std::cout<<"Host -> Device: " << h2dElapsed.count() << " ms" << std::endl;
  std::cout<<"2D2DKernel: " << 2D2DElapsed.count() << " ms" << std::endl;
  std::cout<<"Device -> Host: " << d2hElapsed.count() << " ms" << std::endl;
  std::cout<<"1D1DKernel: " << 1D1DElapsed.count() << " ms" << std::endl;
  std::cout<<"2D1DKernel: " << 2D1DElapsed.count() << " ms" << std::endl;

	// ***

	// validation
	bool isCorrect = true;
	for (int layout = 0 ; layout < NUM_LAYOUTS; layout++) {
		isCorrect = true;
		for (int i = 0; i < MAT_SIZE; i++) {
			if (hC[i] != C[layout][i]) {
				isCorrect = false;
				break;
			}
		}

		switch (layout) {
		case G1D_B1D:
			printf("G1D_B1D");
			break;
		case G2D_B1D:
			printf("G2D_B1D");
			break;
		case G2D_B2D:
			printf("G2D_B2D");
			break;
		}
		if (isCorrect) printf(" kernel works well!\n");
		else printf(" kernel fails to make correct result(s)..\n");
	}

	timer.printTimer();

	SAFE_DELETE(A);
	SAFE_DELETE(B);
	SAFE_DELETE(hC);
	LOOP_I(4){ SAFE_DELETE(C[i]); }
	return 0;
}

/******************************************************************
* Complete following three functions
******************************************************************/

bool MatAddGPU_2D2D(float* _dA, float* _dB, float* _dC)
{
	//** Set the block and grid layout for your 2D2D kernel **//
	dim3 blockDim(32, 32);
	dim3 gridDim(ceil((float)COL_SIZE/ blockDIm.x), ceil((float)ROW_SIZE/blockDim.y));
  MatAdd_G2D_B2D<<<gridDim, blockDIm>>> (A,B,C, ROW_SIZE, COL_SIZE);
	/***********************************************************/

	return kernelCall(_dA, _dB, _dC, ROW_SIZE, COL_SIZE
		, ThreadLayout::G2D_B2D, gridDim, blockDim);
}

bool MatAddGPU_1D1D(float* _dA, float* _dB, float* _dC)
{
	//** Set the block and grid layout for your 1D1D kernel **//
	dim3 blockDim(32);
	dim3 gridDim(ceil((float)COL_SIZE/blockDim.x));
  MatAdd_GID_BID<<<gridDim, blockDim>>>(A,B,C, ROW_SIZE, COL_SIZE);
	/***********************************************************/

	return kernelCall(_dA, _dB, _dC, ROW_SIZE, COL_SIZE
		, ThreadLayout::G1D_B1D, gridDim, blockDim);
}

bool MatAddGPU_2D1D(float* _dA, float* _dB, float* _dC)
{
	//** Set the block and grid layout for your 2D1D kernel **//
	dim3 blockDim( 1 /* block dimenstions */);
	dim3 gridDim( 1 /* grid dimensions */);
	/***********************************************************/

	return kernelCall(_dA, _dB, _dC, ROW_SIZE, COL_SIZE
		, ThreadLayout::G2D_B1D, gridDim, blockDim);

}

SyntaxError: invalid decimal literal (<ipython-input-1-8240878eb975>, line 26)